<table align="left" width=100%>
    <tr><td width="20%">
            <img src="Great_Learning_Logo.jpeg">
        </td>
        <td>
            <div align="center">
                <font color="#21618C" size=8px>
                  <b> Mini Project - CT2 </b>
                </font>
            </div>
        </td>
    </tr>
</table>

## <font color='green'> Group Number:  CT2 Project Group - 5 (SQL) </font> 

### Team Members

<font color='blue' size=4> 1. Vinod A <br>
2. Ramarajan V <br>
3. Francis Alexandre Akshay Rajan </font> 

In [1]:
# pip install mysql-connector-python

In [2]:
import mysql.connector
import pandas as pd

In [3]:
# to be configured
mydb = mysql.connector.connect(
  host="hostname", # hostname of the database
  user="username", # username of the database
  password="password" # password of the database
)

mycursor = mydb.cursor()

In [4]:
# database creation
mycursor.execute("create database buyYourBooks;")

In [5]:
# to use databse
mycursor.execute("use buyYourBooks;")

In [6]:
# Book table creation
query = "Create table Book (Bookid varchar(10) primary key, Bookname varchar(20), Category varchar(20));"
mycursor.execute(query)

In [7]:
# instertion of data in book table

query = [
    "Insert into Book values ('B101', 'Science Revolution', 'Journal');",
"Insert into Book values ('B102', 'Brain Teasers', 'Aptitude');" ,
"Insert into Book values ('B103', 'India Today', 'Magazine');" ,
"Insert into Book values ('B104', 'Tech World', 'Journal');",
"Insert into Book values ('B105', 'Bizz world', 'Magazine');", 
"Insert into Book values ('B106', 'The Quests', 'Aptitude');"
]

In [8]:
for i in query:
    mycursor.execute(i)
    mydb.commit()

In [9]:
mycursor.execute("select * from book;")
mycursor.fetchall()

[('B101', 'Science Revolution', 'Journal'),
 ('B102', 'Brain Teasers', 'Aptitude'),
 ('B103', 'India Today', 'Magazine'),
 ('B104', 'Tech World', 'Journal'),
 ('B105', 'Bizz world', 'Magazine'),
 ('B106', 'The Quests', 'Aptitude')]

In [10]:
# Customer table creation
query = "create table customer(custid varchar(10) primary key,custname varchar(20));"
mycursor.execute(query)

In [11]:
# instertion of data in customer table

query = [
    "Insert into customer values('C101', 'Jack'); ",
"Insert into customer values('C102', 'Anne'); ",
"Insert into customer values('C103', 'Jane'); ",
"Insert into customer values('C104', 'Maria');",
]

In [12]:
for i in query:
    mycursor.execute(i)
    mydb.commit()

In [13]:
mycursor.execute("select * from customer;")
mycursor.fetchall()

[('C101', 'Jack'), ('C102', 'Anne'), ('C103', 'Jane'), ('C104', 'Maria')]

In [14]:
# Purchase table creation
query = "create table purchase(purchaseid varchar(10) primary key,custid varchar(10) references customer(custid),bookid varchar(10) references book(bookid),purchasedate date );"
mycursor.execute(query)

In [15]:
# instertion of data in purchase table

query = [
    "Insert into purchase values ('P201','C101', 'B102','2019-12-12');",
"Insert into purchase values ('P202','C102','B103','2019-11-25');",
"Insert into purchase values ('P203','C103', 'B104','2019-12-12');",
"Insert into purchase values ('P204','C104', 'B105','2019-11-25');",
"Insert into purchase values ('P205','C101', 'B101','2019-12-11');",
"Insert into purchase values ('P206','C101', 'B106','2019-12-12');",
]

In [16]:
for i in query:
    mycursor.execute(i)
    mydb.commit()

In [17]:
mycursor.execute("select * from purchase;")
mycursor.fetchall()

[('P201', 'C101', 'B102', datetime.date(2019, 12, 12)),
 ('P202', 'C102', 'B103', datetime.date(2019, 11, 25)),
 ('P203', 'C103', 'B104', datetime.date(2019, 12, 12)),
 ('P204', 'C104', 'B105', datetime.date(2019, 11, 25)),
 ('P205', 'C101', 'B101', datetime.date(2019, 12, 11)),
 ('P206', 'C101', 'B106', datetime.date(2019, 12, 12))]

<br>
<br>
<font size=4>
    <b>Requirement 1 :</b>
</font>
<font size=3>
Identify the purchase details of books, that are purchased exactly on different dates by the same customer(s). Write a query to display customer’s id and number of such purchases to be displayed as BOOKS for the identified purchase details.</font>

In [18]:
query1 = """select custid, count(distinct purchasedate) AS num_of_books
from purchase
group by custid
having count(distinct purchasedate) > 1;"""

In [19]:
mycursor.execute(query1)
pd.DataFrame(mycursor.fetchall(), columns=['Customer ID', 'Number of books'])

,Customer ID,Number of books
0,C101,2


<br>
<br>
<font size=4>
    <b>Requirement 2 :</b>
</font>
<font size=3>
Identify the purchase details of books, where the books of the same category are purchased by different customers on different dates. Write a query to display customer’s id and title of the book for the identified purchase details.
</font>

In [20]:
query2 = """select p.custid, b.Bookname
from purchase p
join customer c on p.custid = c.custid
join Book b on p.Bookid = b.Bookid
where b.Category in (
    select b1.Category
    from Book b1
    join purchase p1 on b1.Bookid = p1.Bookid
    group by b1.Category
    having count(distinct p1.custid) > 1 and count(distinct p1.purchasedate) > 1
)
order by p.custid;"""

In [21]:
mycursor.execute(query2)
pd.DataFrame(mycursor.fetchall(), columns=['Customer ID', 'Title of the book'])

,Customer ID,Title of the book
0,C101,Science Revolution
1,C103,Tech World


<br>
<br>
<font size=4>
    <b>Requirement 3 :</b>
</font>
<font size=3>
Identify the purchase details of books, where the book is purchased on the same date, exactly on the date Anne has purchased the book. Write a query to display customer’s name and title of the book for the identified purchase details. Do NOT display details of Anne in the query result.</font>

In [22]:
query3 = """select c.custname, b.Bookname
from customer c
join purchase p on c.custid = p.custid
join Book b on p.Bookid = b.Bookid
where p.purchasedate = (
    select purchasedate
    from purchase
    where custid = (
        select custid
        from customer
        where custname = 'Anne'
    )
)
and c.custname != 'Anne';"""

In [23]:
mycursor.execute(query3)
pd.DataFrame(mycursor.fetchall(), columns=['Customer Name', 'Title of the book'])

,Customer Name,Title of the book
0,Maria,Bizz world


In [24]:
mydb.close()